In [1]:
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
import seaborn as sns
import models
from models.evaluation import cross_validation_predict, cross_validation_score
from sklearn.metrics import accuracy_score
from utils import load_datasets, load_target, save_submission
import json
sns.set_style('darkgrid')
%matplotlib inline


In [2]:
config = json.load(open('./config/default.json'))
# X_train, X_test = load_datasets(["Age", "AgeSplit", "EducationNum"])
# X_train, X_test = load_datasets(config['features'])
X_train, X_test = load_datasets(['Native'])
y_train = load_target('target')


In [3]:
X_train

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
1,-9.536743e-07,-9.536743e-06,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000043,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
2,-9.536743e-07,-1.536743e-06,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,-0.000007,...,0.000042,0.000084,0.000048,0.000081,0.000106,0.000072,1.046326e-05,0.000008,0.000019,1.046326e-06
3,4.632568e-08,-5.536743e-06,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,-0.000007,...,0.000068,0.000151,0.000100,0.000180,0.000202,0.000153,2.146326e-05,0.000015,0.000046,-9.536743e-07
4,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123988,-9.536743e-07,1.046326e-05,0.000057,0.000006,0.000080,0.000260,0.000140,0.000096,-0.000033,-0.000010,...,0.000027,0.000014,0.000057,0.000044,0.000014,-0.000033,-9.536743e-06,0.000010,-0.000010,-9.536743e-07
123989,-9.536743e-07,-4.536743e-06,0.000002,0.000042,0.000066,0.000018,0.000057,0.000031,-0.000002,-0.000007,...,0.000049,0.000122,0.000053,0.000120,0.000133,0.000088,1.146326e-05,0.000010,0.000027,4.632568e-08
123990,-9.536743e-07,-4.536743e-06,-0.000019,-0.000053,-0.000092,-0.000129,-0.000099,-0.000045,-0.000026,-0.000009,...,0.000090,0.000358,0.000328,0.000310,0.000545,0.000244,5.546326e-05,0.000073,0.000075,4.632568e-08
123991,-9.536743e-07,4.632568e-07,-0.000003,0.000176,0.000350,0.000290,0.000200,0.000206,-0.000023,-0.000010,...,0.000017,0.000124,0.000057,0.000104,0.000144,0.000027,4.632568e-07,0.000060,0.000020,-9.536743e-07


In [4]:
# rf = models.RandomForest(
#     json.load(open('config/RandomForest.json')))
model = models.Lgbm({'objective': 'multiclass',
                    'num_class': 10,
                    "verbosity":-1})
y_oofs, y_preds = cross_validation_predict(
    model, X_train, y_train, X_test, n_splits=5)


/Users/yutahirai/opt/anaconda3/envs/tensorflow26/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yutahirai/opt/anaconda3/envs/tensorflow26/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yutahirai/opt/anaconda3/envs/tensorflow26/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback 

In [5]:
score = accuracy_score(y_train, np.argmax(y_oofs, axis=1))
print(score)


0.9619575298605566


In [6]:
save_submission(np.argmax(y_preds, axis=1), score, 'target', "lgbm-Native")